In [3]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langgraph.graph import StateGraph
from typing import TypedDict, Optional
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
import os

In [4]:
# --- .env 에서 OPENAI API 키 불러오기 ---
load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")

# --- LLM 객체 생성 ---
llm = ChatOpenAI(model="gpt-4o", openai_api_key=openai_api_key)

In [ ]:
# --- 🔎 상태 정의 ---
class ProposalState(TypedDict, total=False):
    brand_name: Optional[str]
    brand_info: Optional[str]
    client_needs: Optional[str]
    recommended_media: Optional[str]
    previous_campaigns: Optional[str]
    proposal_text: Optional[str]
    proposal_file_path: Optional[str]

AgentState = ProposalState

In [ ]:
# --- 🗄️ Tool 구성 (여기서는 예시 Stub 형태로 Tool 설정) ---
# 실제 서비스에서는 아래 Tool들을 LangChain Toolkit으로 구현
def db_query_tool(query: str) -> str:
    return f"[DB QUERY RESULT for: {query}]"

def web_search_tool(query: str) -> str:
    return f"[WEB SEARCH RESULT for: {query}]"

def vectordb_search_tool(query: str) -> str:
    return f"[VECTOR DB RESULT for: {query}]"

In [ ]:
# --- Node 1: 브랜드 정보 + 고객 요구사항 분석 ---
def analyze_brand_and_needs(state: ProposalState):
    brand_name = state["brand_name"]
    # 브랜드 + 영업 기록 조회
    brand_query = f"""
    SELECT * FROM brand WHERE brand_name = '{brand_name}';
    SELECT * FROM sales_log WHERE brand_name = '{brand_name}' ORDER BY contact_time DESC LIMIT 1;
    """
    brand_info = db_query_tool(brand_query)

    # 고객 요구사항 추출
    needs_query = f"""
    SELECT client_needs_summary FROM sales_log 
    WHERE brand_name = '{brand_name}' ORDER BY contact_time DESC LIMIT 1;
    """
    client_needs = db_query_tool(needs_query)

    return {**state, "brand_info": brand_info, "client_needs": client_needs}

# --- Node 2: 매체 추천 및 매칭 ---
def recommend_media(state: ProposalState):
    client_needs = state.get("client_needs") or ""
    db_results = db_query_tool("SELECT * FROM media WHERE quantity > 0;")
    vector_results = vectordb_search_tool(client_needs)
    web_results = web_search_tool(client_needs + "에 적합한 옥외 광고 매체 추천")

    combined = f"""
    [DB 추천]\n{db_results}\n
    [VectorDB 추천]\n{vector_results}\n
    [웹 검색 추천]\n{web_results}
    """
    return {**state, "recommended_media": combined}

# --- Node 3: 유사 집행 사례 조회 ---
def retrieve_previous_campaigns(state: ProposalState):
    brand_name = state["brand_name"]
    query = f"""
    SELECT c.*, cm.*, m.media_name FROM campaign c
    JOIN campaign_media cm ON c.campaign_id = cm.campaign_id
    JOIN media m ON cm.media_id = m.media_id
    WHERE c.brand_id = (SELECT brand_id FROM brand WHERE brand_name = '{brand_name}');
    """
    previous_campaigns = db_query_tool(query)
    return {**state, "previous_campaigns": previous_campaigns}

# --- Node 4: 제안서 생성 (Word 파일 포함) ---
def generate_proposal(state: ProposalState):
    prompt = ChatPromptTemplate.from_template("""
    브랜드명: {brand_name}
    브랜드 정보: {brand_info}
    고객 요구사항: {client_needs}
    유사 집행 사례: {previous_campaigns}
    추천 매체: {recommended_media}

    위 정보를 바탕으로 고객에게 보낼 옥외 광고 제안서를 작성하세요.
    """)

    chain = prompt | llm
    proposal = chain.invoke(state).content

    # Word 파일 생성
    doc = Document()
    doc.add_heading(f"{state['brand_name']} 옥외 광고 제안서", level=1)
    doc.add_paragraph(proposal)

    file_name = f"{state['brand_name']}_제안서.docx"
    doc.save(file_name)

    return {**state, "proposal_text": proposal, "proposal_file_path": file_name}


In [ ]:
# --- 🔗 그래프 구성 ---
graph = StateGraph(ProposalState)

graph.add_node("AnalyzeBrandAndNeeds", analyze_brand_and_needs)
graph.add_node("RecommendMedia", recommend_media)
graph.add_node("RetrievePreviousCampaigns", retrieve_previous_campaigns)
graph.add_node("GenerateProposal", generate_proposal)

graph.set_entry_point("AnalyzeBrandAndNeeds")
graph.add_edge("AnalyzeBrandAndNeeds", "RecommendMedia")
graph.add_edge("RecommendMedia", "RetrievePreviousCampaigns")
graph.add_edge("RetrievePreviousCampaigns", "GenerateProposal")
graph.set_finish_point("GenerateProposal")

proposal_graph = graph.compile()

In [ ]:
# --- 🚀 실행 예시 ---
initial_state = {
    "brand_name": "라네즈",
}

final_state = proposal_graph.invoke(initial_state)

print("✅ 최종 제안서:\n")
print(final_state["proposal_text"])
print(f"📄 제안서 Word 파일 경로: {final_state['proposal_file_path']}")

✅ 최종 제안서:
content='**라네즈 옥외 광고 제안서**\n\n---\n\n**브랜드명:** 라네즈  \n**제안서 수신인:** [고객명]\n\n---\n\n**브랜드 정보:**\n\n라네즈는 혁신적인 피부 과학을 기반으로 한 성분과 기술을 활용하여 수분 관리 솔루션을 제공하는 대표적인 스킨케어 브랜드입니다. 고객의 피부 상태와 필요에 따른 맞춤형 제품을 개발하여 다양한 피부 고민을 해결하고자 합니다.\n\n**고객 요구사항:**\n\n최근 고객들은 피부 수분 관리에 대한 관심이 증가하고 있으며, 이에 따라 라네즈의 수분 라인 제품에 대한 적극적인 홍보가 필요합니다. 특히, 사회 활동이 많은 20-30대 여성에게 브랜드의 인지도를 높이는 것이 중요합니다.\n\n**이전 캠페인 요약:**\n\n- **뷰티박람회 참여**: 라네즈는 다양한 뷰티박람회에서 제품을 직접 경험할 수 있는 부스를 운영하여 높은 호응을 받았습니다.\n- **온라인 광고**: 주로 SNS 플랫폼을 통해 브랜드 메시지를 전달하였으며, 이는 온라인 상에서 긍정적인 피드백을 얻었습니다.\n- **미디어 협업**: 국내 주요 뷰티 유튜버와 협업하여 제품 리뷰와 튜토리얼을 진행하였습니다.\n\n**추천 매체:**\n\n- **버스 쉘터 광고**: 도심 속 다양한 이동 경로에 위치한 버스 쉘터는 젊은 층의 높은 주목도를 자랑합니다.\n- **지하철 내부 광고**: 출퇴근 시간대 밀집된 인구에게 제품 메시지를 효과적으로 전달할 수 있으며 브랜드 인지도를 높이는 데 유리합니다.\n- **디지털 스크린 광고**: 강남, 홍대 등 트렌드에 민감한 지역의 대형 스크린을 활용한 광고 송출로 브랜드 이미지를 부각시킬 수 있습니다.\n\n**추천 매체의 장점:**\n\n- **실시간 노출**: 도심의 주요 지점에서 많은 사람에게 반복적으로 노출할 수 있습니다.\n- **타겟 고객 접근성**: 주요 상권 및 다중 이용 시설에 위치하여 20-30대 여성들의 접근성이 높습니다.\n- **다이내믹한 콘텐츠**: 디